In [5]:
# Librerias

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from tabulate import tabulate #Ni idea de esta libreria
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
#sns.set_style()
#import plotly.graph_objects as go
import plotly.express as px
#import calendar
#import plotly.io as pio

In [7]:
#Reading file and checking columns

path = "D:\\Proyects\\IncidenciaDelictivaMexico\\Data\\"
idedo = pd.read_csv(path + "IDEFC_NM_ago23.csv", encoding='iso-8859-1') #incidencia delictiva estatal
idmun = pd.read_csv(path + "IDM_NM_ago23.csv", encoding='iso-8859-1') #incidencia delictiva municipal
idfc = pd.read_csv(path + "IDVFC_NM_ago23.csv", encoding='iso-8859-1') #incidencia fuera fuero comun

## Analisis Incidencia Delictiva por Estado (IDEFC & IDVFC)

#### Cleaning dataframes to append

In [70]:
# Preview 

df = idedo.replace(to_replace= np.nan, value=0)
df = idedo.replace(to_replace= np.nan, value=0)
#df.isna().sum()
#print('Incidencia estatal',df.shape)
df1= idfc.replace(to_replace= np.nan, value=0)
df1 = idfc.replace(to_replace= np.nan, value=0)
#print('Fuero comun',df1.shape)
#df.info()

In [82]:
# Filling nan values with 0, changing values to int64

#df1 - fuero comun select change columns to int64
df1 = df1.replace(to_replace= np.nan, value=0)
numeric_columns =  df1.loc[:,["Año","Clave_Ent","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]]
for col in numeric_columns:
    df1[col] = pd.to_numeric(df1[col],errors='coerce').astype(np.int64)
#df1.info()

#df - incidencia estatal change columns to int64
df = df.replace(to_replace= np.nan, value=0)
numeric_columns1 =  df.loc[:,["Año","Clave_Ent","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]]
for col in numeric_columns1:
    df[col] = pd.to_numeric(df[col],errors='coerce').astype(np.int64)
#df.info()
print('IncidenciaEstatal',df.shape)
print('IncidenciaFuero',df1.shape)
print('IncidenciaTotal:',28224+66240)

IncidenciaEstatal (28224, 19)
IncidenciaFuero (66240, 21)
IncidenciaTotal: 94464


In [83]:
#Selecting columns to append on both dataframes

appendf = df[["Año","Entidad","Bien jurídico afectado","Tipo de delito","Subtipo de delito","Modalidad","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]]
#appendf.head()
pd.DataFrame(appendf)
print('IncidenciaEstatal',appendf.shape)
appendf1 = df1[["Año","Entidad","Bien jurídico afectado","Tipo de delito","Subtipo de delito","Modalidad","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]]
#appendf1
pd.DataFrame(appendf1)
print('IncidenciaFueroComun',appendf1.shape)

IncidenciaEstatal (28224, 18)
IncidenciaFueroComun (66240, 18)


In [84]:
#Unir los dataframes

appended_df = pd.concat([appendf, appendf1], ignore_index=True)
#appended_df.shape #si se unio
#Columna recuento
appended_df['Recuento'] = appended_df["Enero"] + appended_df["Febrero"] + appended_df["Marzo"] + appended_df["Abril"] + appended_df["Mayo"] + appended_df["Junio"] + appended_df["Julio"] + appended_df["Agosto"] + appended_df["Septiembre"] + appended_df["Octubre"] + appended_df["Noviembre"] + appended_df["Diciembre"] 
appended_df = pd.DataFrame(appended_df)
print('Total:',appended_df.shape)
#appended_df.tail()

Total: (94464, 19)


In [91]:
#Analisis
#Entidad
entidad_recuento = appended_df.groupby('Entidad')['Recuento'].sum().reset_index()
fig1 = px.bar(entidad_recuento, x='Entidad', y='Recuento', text='Recuento', text_auto='.2s',
              color='Recuento', labels={'Recuento': 'Recuento por estados'},
              color_continuous_scale='Viridis')  
fig1.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5,
                    opacity=0.8, textposition='inside') 
fig1.update_layout(title_text='Delitos por estado', yaxis_title='Entidades',
                   xaxis_title='Recuento', height=500, xaxis={'categoryorder':'total descending'})
fig1.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
#fig1.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig1.show()

#Bien juridico afectado
bienj_recuento = appended_df.groupby('Bien jurídico afectado')['Recuento'].sum().reset_index()
fig = px.pie(bienj_recuento, names="Bien jurídico afectado", values="Recuento", hole=0.3)

fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(title_text='Bien Jurídico Afectado', yaxis_title='',
                   xaxis_title='', height=500, xaxis={'categoryorder':'total descending'})
fig.show()

# #Bien juridico afectado
# bienj_recuento = appended_df.groupby('Bien jurídico afectado')['Recuento'].sum().reset_index()
# #df = px.data.gapminder().query("year == 2007").query("continent == 'Americas'")
# fig = px.pie(bienj_recuento, values='Recuento', names='Bien jurídico afectado',
#              title='',
#              hover_data=['Recuento'])
# fig.update_traces(textposition='outside', textinfo='percent+label')
# fig.update_layout(title_text='Bien Jurídico Afectado', yaxis_title='',
#                    xaxis_title='', height=500, xaxis={'categoryorder':'total descending'})
# fig.show()

#Desgloce Sunburst
fig2 = px.sunburst(appended_df, path=['Bien jurídico afectado', 'Tipo de delito', 'Subtipo de delito','Modalidad'], values='Recuento') #,color="Recuento")
#fig.update_layout(margin = dict(t = 10, l = 10, r = 10, b = 10))#, textinfo='percent+label') #PARA HACERLO MÁS GRANDE
fig2.update_layout(title_text='Analisis por niveles de detalle en el bien juridico afectado')
fig2.update_layout(uniformtext=dict(minsize=10, mode='hide')) #PARA AJUSTAR AL TEXTO
# fig.update_layout(title_text='Bien Jurídico Afectado', yaxis_title='',
#                    xaxis_title='', height=500, xaxis={'categoryorder':'total descending'})
#fig.update_traces(textposition='outside', textinfo='percent+label')
fig2.show()

#Año analisis
año_recuento = appended_df.groupby('Año')['Recuento'].sum().reset_index()
fig3 = px.bar(año_recuento, x='Año', y='Recuento', text='Recuento', text_auto='.2s',
              color='Recuento', labels={'Recuento': 'Recuento por año'},
              color_continuous_scale='tempo')  
fig3.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5,
                    opacity=0.8, textposition='inside') 
fig3.update_layout(title_text='Delitos por año', yaxis_title='Años',
                   xaxis_title='Recuento', height=500, xaxis={'categoryorder':'total descending'})
fig3.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig3.show()

#### Avance hasta aquí.
Pendientes:
* Agregar lat y log, usar pyplot geomap.
* Ver la forma de asignar a cada municipio y estado las lat y log. Usar la base de datos del INEGI.
* Hacer analisis con unicamente la base de datos de fuero común.

In [89]:
#Desgloce Sunburst

fig2 = px.sunburst(appended_df, path=['Bien jurídico afectado', 'Tipo de delito', 'Subtipo de delito','Modalidad'], values='Recuento') #,color="Recuento")
#fig.update_layout(margin = dict(t = 10, l = 10, r = 10, b = 10))#, textinfo='percent+label') #PARA HACERLO MÁS GRANDE
fig2.update_layout(title_text='Analisis por niveles de detalle en el bien juridico afectado')
fig2.update_layout(uniformtext=dict(minsize=10, mode='hide')) #PARA AJUSTAR AL TEXTO
# fig.update_layout(title_text='Bien Jurídico Afectado', yaxis_title='',
#                    xaxis_title='', height=500, xaxis={'categoryorder':'total descending'})
#fig.update_traces(textposition='outside', textinfo='percent+label')
fig2.show()

In [ ]:
#Desgloce Sunburst
fig = px.sunburst(df_1, path=['Bien jurídico afectado', 'Tipo de delito', 'Subtipo de delito','Modalidad'], values='Recuento') #,color="Recuento")
#fig.update_layout(margin = dict(t = 10, l = 10, r = 10, b = 10))#, textinfo='percent+label') #PARA HACERLO MÁS GRANDE
fig.update_layout(title_text='Analisis por niveles de detalle en el bien juridico afectado')
fig.update_layout(uniformtext=dict(minsize=10, mode='hide')) #PARA AJUSTAR AL TEXTO
#fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [68]:
#Bien juridico afectado
bienj_recuento = appended_df.groupby('Bien jurídico afectado')['Recuento'].sum().reset_index()
fig = px.pie(bienj_recuento, names="Bien jurídico afectado", values="Recuento", hole=0.3)

fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(title_text='Bien Jurídico Afectado', yaxis_title='',
                   xaxis_title='', height=500, xaxis={'categoryorder':'total descending'})
fig.show()

In [9]:
#df = idedo.replace(to_replace= np.nan, value=0)
numeric_columns =  df.loc[:,["Año","Clave_Ent","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col],errors='coerce').astype(np.int64)
#df.tail(10)
### Creo que esta demás la otra variable, checar y hacer test si es el caso.

In [10]:
#Columna recuento
df['Recuento'] = df["Enero"] + df["Febrero"] + df["Marzo"] + df["Abril"] + df["Mayo"] + df["Junio"] + df["Julio"] + df["Agosto"] + df["Septiembre"] + df["Octubre"] + df["Noviembre"] + df["Diciembre"] 
prueba = pd.DataFrame(df)

# Edo_recuento = df.groupby('Entidad')['Recuento'].sum().reset_index()
# print(Edo_recuento)

In [11]:
#Recunto edo
edo_recuento = df.groupby('Entidad')['Recuento'].sum().reset_index()
fig1 = px.bar(edo_recuento, x='Entidad', y='Recuento', text='Recuento', text_auto='.2s',
              color='Recuento', labels={'Recuento': 'Recuento por estados'},
              color_continuous_scale='Viridis')  
fig1.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5,
                    opacity=0.8, textposition='inside') 
fig1.update_layout(title_text='Incidencia delictiva por estado', yaxis_title='Entidades',
                   xaxis_title='Recuento', height=500, xaxis={'categoryorder':'total descending'})
fig1.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
#fig1.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig1.show()

In [12]:
#Recuento bien juridico afectado

bien_recuento = df.groupby('Bien jurídico afectado')['Recuento'].sum().reset_index()

#df = px.data.gapminder().query("year == 2007").query("continent == 'Americas'")
fig = px.pie(bien_recuento, values='Recuento', names='Bien jurídico afectado',
             title='',
             hover_data=['Recuento'])
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(title_text='Bien Jurídico Afectado', yaxis_title='',
                   xaxis_title='', height=500, xaxis={'categoryorder':'total descending'})
fig.show()




In [13]:
df_1 = df[["Año","Entidad","Bien jurídico afectado","Tipo de delito","Subtipo de delito","Modalidad","Recuento"]]
df_1 = pd.DataFrame(df_1)
df_1.tail()

,Año,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Recuento
28219,2023,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Falsificación,Falsificación,Falsificación,69
28220,2023,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Contra el medio ambiente,Contra el medio ambiente,Contra el medio ambiente,4
28221,2023,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Delitos cometidos por servidores públicos,Delitos cometidos por servidores públicos,Delitos cometidos por servidores públicos,181
28222,2023,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Electorales,Electorales,Electorales,5
28223,2023,Zacatecas,Otros bienes jurídicos afectados (del fuero co...,Otros delitos del Fuero Común,Otros delitos del Fuero Común,Otros delitos del Fuero Común,1423


In [14]:
fig = px.sunburst(df_1, path=['Bien jurídico afectado', 'Tipo de delito', 'Subtipo de delito','Modalidad'], values='Recuento') #,color="Recuento")
#fig.update_layout(margin = dict(t = 10, l = 10, r = 10, b = 10))#, textinfo='percent+label') #PARA HACERLO MÁS GRANDE
fig.update_layout(title_text='Analisis por niveles de detalle en el bien juridico afectado')
fig.update_layout(uniformtext=dict(minsize=10, mode='hide')) #PARA AJUSTAR AL TEXTO
#fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [15]:
fig = px.sunburst(df_1, path=['Bien jurídico afectado', 'Tipo de delito'], values='Recuento',maxdepth=2)

# fig.update_layout(margin = dict(t = 10, l = 10, r = 10, b = 10))    #, textinfo='percent+label') #Lo hace mas grande
#fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(title_text='Desgloce Bien Jurídico Afectado')
fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.show()

fig1 = px.sunburst(df_1, path=['Subtipo de delito','Modalidad'], values='Recuento', color= "Recuento", color_continuous_scale='RdBu' ) #Color lle da el gradiente
#fig1.update_layout(margin = dict(t = 10, l = 10, r = 10, b = 10))#, textinfo='percent+label')
fig1.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig1.update_layout(title_text='Subtipo de delito')
#fig.update_traces(textposition='outside', textinfo='percent+label')
fig1.show()

In [16]:
#print(bien_recuento) simon
fig1 = px.bar(bien_recuento, x='Bien jurídico afectado', y='Recuento', text='Recuento', text_auto='.2s',
              color='Recuento', labels={'Recuento': 'Recuento por estados'},
              color_continuous_scale='mint')  
fig1.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5,
                    opacity=0.8, textposition='inside') 
fig1.update_layout(title_text='Incidencia delictiva por bien jurídico afectado', yaxis_title='Bien jurídico afectado',
                   xaxis_title='Recuento', height=500, xaxis={'categoryorder':'total descending'})
fig1.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
#fig1.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig1.show()

## Analisis Incidencia Delictiva Fuera Común

In [17]:
df2 = idfc.replace(to_replace= np.nan, value=0)
df2.isna().sum()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66240 entries, 0 to 66239
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     66240 non-null  int64  
 1   Clave_Ent               66240 non-null  int64  
 2   Entidad                 66240 non-null  object 
 3   Bien jurídico afectado  66240 non-null  object 
 4   Tipo de delito          66240 non-null  object 
 5   Subtipo de delito       66240 non-null  object 
 6   Modalidad               66240 non-null  object 
 7   Sexo                    66240 non-null  object 
 8   Rango de edad           66240 non-null  object 
 9   Enero                   66240 non-null  int64  
 10  Febrero                 66240 non-null  int64  
 11  Marzo                   66240 non-null  int64  
 12  Abril                   66240 non-null  int64  
 13  Mayo                    66240 non-null  int64  
 14  Junio                   66240 non-null

In [18]:
df2 = df2.replace(to_replace= np.nan, value=0)
numeric_columns =  df2.loc[:,["Año","Clave_Ent","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]]
for col in numeric_columns:
    df2[col] = pd.to_numeric(df2[col],errors='coerce').astype(np.int64)

In [24]:
#Columna recuento
df2['Recuento'] = df2["Enero"] + df2["Febrero"] + df2["Marzo"] + df2["Abril"] + df2["Mayo"] + df2["Junio"] + df2["Julio"] + df2["Agosto"] + df2["Septiembre"] + df2["Octubre"] + df2["Noviembre"] + df2["Diciembre"] 
prueba2 = pd.DataFrame(df2)
prueba2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66240 entries, 0 to 66239
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Año                     66240 non-null  int64 
 1   Clave_Ent               66240 non-null  int64 
 2   Entidad                 66240 non-null  object
 3   Bien jurídico afectado  66240 non-null  object
 4   Tipo de delito          66240 non-null  object
 5   Subtipo de delito       66240 non-null  object
 6   Modalidad               66240 non-null  object
 7   Sexo                    66240 non-null  object
 8   Rango de edad           66240 non-null  object
 9   Enero                   66240 non-null  int64 
 10  Febrero                 66240 non-null  int64 
 11  Marzo                   66240 non-null  int64 
 12  Abril                   66240 non-null  int64 
 13  Mayo                    66240 non-null  int64 
 14  Junio                   66240 non-null  int64 
 15  Ju

In [26]:
prueba2.tail()
fig = px.sunburst(prueba2, path=['Sexo','Bien jurídico afectado', 'Tipo de delito', 'Subtipo de delito','Modalidad',"Rango de edad"], values='Recuento') #,color="Recuento")
#fig.update_layout(margin = dict(t = 10, l = 10, r = 10, b = 10))#, textinfo='percent+label') #PARA HACERLO MÁS GRANDE
fig.update_layout(title_text='Analisis por niveles de detalle fuero comun')
fig.update_layout(uniformtext=dict(minsize=10, mode='hide')) #PARA AJUSTAR AL TEXTO
#fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()